<a href="https://colab.research.google.com/github/seenu-g/school_of_ai/blob/master/c2/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn #Base class for all neural network modules.
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module): 
    def __init__(self):
        super(Net, self).__init__()
        
        #Applies a 2D convolution over an input signal composed of several input planes.  https://pytorch.org/docs/stable/nn.html#conv2d
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
            # in_channels (python:int) – Number of channels in the input image
            # out_channels (python:int) – Number of channels produced by the convolution
            # kernel_size (python:int or tuple) – Size of the convolving kernel
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        
        #Applies a 2D max pooling over an input signal composed of several input planes.
        #torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
            #kernel_size – the size of the window to take a max over
            #stride – the stride of the window. Default value is kernel_size
            #padding – implicit zero padding to be added on both sides
            #dilation – a parameter that controls the stride of elements in the window
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(256, 512, 3)
        self.conv6 = nn.Conv2d(512, 1024, 3)
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        #Applies the rectified linear unit function element-wise:
        #ReLU(x)=max(0,x) #Input: (N, *)(N,∗) #Output: (N, *)(N,∗) 
        #torch.nn.ReLU(inplace=False) #inplace – can optionally do the operation in-place. 
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        
        # Returns a new tensor with the same data as the self tensor but of a different shape.
        x = x.view(-1, 10) #shape (torch.Size or python:int...) – the desired size

        #Applies a softmax followed by a logarithm. #equivalent to log(softmax(x))
        # This function uses an alternative formulation to compute the output and gradient correctly
        return F.log_softmax(x)

In [20]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()  #Returns a bool indicating if CUDA is currently available.
device = torch.device("cuda" if use_cuda else "cpu") #Context-manager that changes the selected device.
model = Net().to(device) # recursively go over all modules and convert their parameters and buffers to CUDA tensors:
#summary(your_model, input_size=(channels, H, W))
# view the visualization of the model which is very helpful while debugging your network.
summary(model, input_size=(1, 28, 28)) #model: pytorch model object

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1) #seed the RNG for all devices (both CPU and CUDA)
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
#Dataloader Combines a dataset and a sampler, and provides an iterable over the given dataset.
#supports both map-style and iterable-style datasets with single- or multi-process loading, customizing loading order and optional automatic batching (collation) and memory pinning.
train_loader = torch.utils.data.DataLoader(   #dataset from which to load the data.
    datasets.MNIST('../data',                 # If true, downloads the dataset and puts it in root directory. If downloaded, it is not downloaded again.           
                   train=True, download=True, # If True, creates dataset from ``training.pt``, otherwise from ``test.pt``
                   transform=transforms.Compose( # A function/transform that  takes in an PIL image and returns a transformed version
                    [transforms.ToTensor(),
                     transforms.Normalize((0.1307,), (0.3081,))]
                   )),
    batch_size=batch_size, shuffle=True, **kwargs) #batch_size (python:int, optional) – how many samples per batch to load (default: 1).

test_loader = torch.utils.data.DataLoader(   #dataset from which to load the data.
    datasets.MNIST('../data', 
                   train=False, 
                   transform=transforms.Compose(
                   [transforms.ToTensor(),                       #Convert a PIL Image or numpy.ndarray to tensor.
                    transforms.Normalize((0.1307,), (0.3081,))]   # Normalize a tensor image with mean and standard deviation. 
                        # Given mean: (M1,...,Mn) and std: (S1,..,Sn) for n channels, this transform will normalize each channel of the input 
                  )),
    batch_size=batch_size, shuffle=True, **kwargs) #shuffle (bool, optional) – set to True to have the data reshuffled at every epoch (default: False).


In [0]:
from tqdm import tqdm  # Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable)
def train(model, device, train_loader, optimizer, epoch):
    model.train() #By default all the modules are initialized to train mode (self.training = True).
                  #model.train() tells your model that you are training the model. model knows it has to learn the layers
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() # zero_grad clears old gradients from the last step (otherwise you’d just accumulate the gradients from all loss.backward() calls)
        output = model(data)
        loss = F.nll_loss(output, target) #The negative log likelihood loss. nll_loss(input, target, weight=None, size_average=None, ignore_index=-100, reduce=None, reduction='mean')
        loss.backward() # computes the derivative of the loss w.r.t. the parameters (or anything requiring gradients) using backpropagation.
        optimizer.step() # causes the optimizer to take a step based on the gradients of the parameters
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval() # when we use model.eval() it indicates the model that nothing new is to be learnt and the model is used for testing.
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
# optim is a a package implementing various optimization algorithms.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #Implements stochastic gradient descent

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)